<a href="https://colab.research.google.com/github/hariPrasadCoder/Nifty-50-EV-EBITDA-valuation-model/blob/main/NIFTY_50_EV_EBITA_valuation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv

In [3]:
main_source = requests.get('https://www.moneycontrol.com/stocks/marketstats/indexcomp.php?optex=NSE&opttopic=indexcomp&index=9').text

In [4]:
parser = BeautifulSoup(main_source, 'lxml')

In [11]:
links = []
table = parser.find("table", class_='tbldata14 bdrtpg')
for tr in table.find_all("tr")[1:]:
  links.append(tr.find_all('a')[0].attrs['href'])
links

['/india/stockpricequote/infrastructuregeneral/adaniportsspecialeconomiczone/MPS',
 '/india/stockpricequote/paintsvarnishes/asianpaints/AP31',
 '/india/stockpricequote/banksprivatesector/axisbank/AB16',
 '/india/stockpricequote/auto23wheelers/bajajauto/BA10',
 '/india/stockpricequote/financeleasinghirepurchase/bajajfinance/BAF',
 '/india/stockpricequote/financeinvestments/bajajfinserv/BF04',
 '/india/stockpricequote/telecommunicationsservice/bhartiairtel/BA08',
 '/india/stockpricequote/refineries/bharatpetroleumcorporation/BPC',
 '/india/stockpricequote/foodprocessing/britanniaindustries/BI',
 '/india/stockpricequote/pharmaceuticals/cipla/C',
 '/india/stockpricequote/miningminerals/coalindia/CI11',
 '/india/stockpricequote/pharmaceuticals/divislaboratories/DL03',
 '/india/stockpricequote/pharmaceuticals/drreddyslaboratories/DRL',
 '/india/stockpricequote/autolcvshcvs/eichermotors/EM',
 '/india/stockpricequote/diversified/grasimindustries/GI01',
 '/india/stockpricequote/computerssoftwar

In [19]:
csv_file = open('ev_ebitda_scraper.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['company', 'target_price', 'entry_price', 'ltp', 'percent_increase_if_buy_now'])

for company in links:
  symbol = company.split('/')[-1]
  company_name = company.split('/')[-2]
  s1 = 'https://www.moneycontrol.com/financials/'+ company_name + '/consolidated-ratiosVI/' + symbol + '#' + symbol
  s2 = 'https://www.moneycontrol.com/financials/'+ company_name + '/consolidated-balance-sheetVI/' + symbol + '#' + symbol
  s3 = 'https://www.moneycontrol.com/' + company

  try:
    source = requests.get(s1).text
    soup = BeautifulSoup(source, 'lxml')
    tabs = []
    table = soup.find("table", class_='mctable1')
    for tr in table.find_all("tr")[1:]:
      tabs.append({
                'Ratio Titles': tr.find_all('td')[0].text,
                'Mar21': tr.find_all('td')[1].text,
                'Mar20': tr.find_all('td')[2].text,
                'Mar19': tr.find_all('td')[3].text,
                'Mar18': tr.find_all('td')[4].text,
                'Mar17': tr.find_all('td')[5].text
                })
      
    df = pd.DataFrame(tabs)
    df = df.set_index('Ratio Titles')

    df2 = df.loc[['Enterprise Value (Cr.)','EV/EBITDA (X)'],:]
    df2['Mar21'] = df2['Mar21'].str.replace(',','').astype('float')
    df2['Mar20'] = df2['Mar20'].str.replace(',','').astype('float')
    df2['Mar19'] = df2['Mar19'].str.replace(',','').astype('float')
    df2['Mar18'] = df2['Mar18'].str.replace(',','').astype('float')
    df2['Mar17'] = df2['Mar17'].str.replace(',','').astype('float')

    ebitda = (df2.iloc[0,:].values/df2.iloc[1,:].values).tolist()

    df2.loc['EBITA'] = ebitda

    pd.set_option('display.float_format', '{:.2f}'.format)

    list_growth = []
    for i in range(len(ebitda)-1):
      growth = ((ebitda[i]-ebitda[i+1])/ebitda[i+1])*100
      list_growth.append(growth)
      i += 1

    avg_growth_in_ebitda = np.mean(list_growth[:3])

    expected_ebitda = ebitda[0]*((100+avg_growth_in_ebitda)/100)

    source2 = requests.get(s2).text
    soup2 = BeautifulSoup(source2, 'lxml')

    tabs = []
    table = soup2.find("table", class_='mctable1')
    for tr in table.find_all("tr")[1:]:
      tabs.append({
                'Balance Sheet': tr.find_all('td')[0].text,
                'Mar21': tr.find_all('td')[1].text,
                'Mar20': tr.find_all('td')[2].text,
                'Mar19': tr.find_all('td')[3].text,
                'Mar18': tr.find_all('td')[4].text,
                'Mar17': tr.find_all('td')[5].text
                })
      
    df_bs = pd.DataFrame(tabs)
    df_bs = df_bs.set_index('Balance Sheet')

    debt = df_bs.loc[['Long Term Borrowings'],:].values.tolist()[0][1]
    debt_final = float(debt.replace(',',''))

    previous_ev_ebita = float(df2.iloc[1,:].values.tolist()[0])

    forecasted_ev = (expected_ebitda*previous_ev_ebita)-debt_final

    source3 = requests.get(s3).text
    soup3 = BeautifulSoup(source3, 'lxml')

    tabs = []
    table = soup3.find("table", class_='peer_tbl')
    for tr in table.find_all("tr")[1:]:
      tabs.append({
                'Company Name': tr.find_all('td')[0].text,
                'ltp': tr.find_all('td')[1].text,
                'MCap': tr.find_all('td')[3].text,
                })
      
    df_mcap = pd.DataFrame(tabs)
    df_mcap = df_mcap.set_index('Company Name')
    
    ltp,mcap = df_mcap.iloc[[0],:].values.tolist()[0]
    ltp_final = float(ltp.replace(',',''))

    mcap_final = float(mcap.replace(',',''))

    no_of_shares = mcap_final/ltp_final

    target_price = forecasted_ev/no_of_shares

    entry_price = target_price*0.75

    incr_now = ((target_price-ltp_final)/ltp_final)*100
  except Exception as e:
    print(company_name)
    print('======>>>>>> ERROR <<<<<<=======')
    print('----------------------------------')
    print()
    continue
  csv_writer.writerow([company_name, target_price, entry_price, ltp_final, incr_now])

  print('Company: ' + company_name)
  print('Target Price: ', target_price)
  print('Entry Price: ', entry_price)
  print('LTP: ', ltp_final)
  print('What if we buy now? ', incr_now)
  print('----------------------------------')
  print()

csv_file.close()

Company: adaniportsspecialeconomiczone
Target Price:  815.5371543298305
Entry Price:  611.6528657473729
LTP:  712.2
What if we buy now?  14.509569549260103
----------------------------------

Company: asianpaints
Target Price:  2911.957653409264
Entry Price:  2183.968240056948
LTP:  3003.9
What if we buy now?  -3.0607658906999626
----------------------------------

axisbank
======>>>>>> ERROR <<<<<<=======
----------------------------------

Company: bajajauto
Target Price:  3701.7456341110505
Entry Price:  2776.309225583288
LTP:  4196.15
What if we buy now?  -11.782332993075775
----------------------------------

Company: bajajfinance
Target Price:  7746.093666038947
Entry Price:  5809.57024952921
LTP:  6075.8
What if we buy now?  27.490925738815413
----------------------------------

Company: bajajfinserv
Target Price:  13495.38067783084
Entry Price:  10121.53550837313
LTP:  12486.6
What if we buy now?  8.078906009889325
----------------------------------

Company: bhartiairtel
Targe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


Company: hdfclifeinsurancecompanylimited
Target Price:  nan
Entry Price:  nan
LTP:  725.95
What if we buy now?  nan
----------------------------------

Company: heromotocorp
Target Price:  2710.22781759164
Entry Price:  2032.67086319373
LTP:  2939.2
What if we buy now?  -7.790289276277897
----------------------------------

Company: hindalcoindustries
Target Price:  362.2699865854713
Entry Price:  271.7024899391035
LTP:  375.9
What if we buy now?  -3.6259679208642357
----------------------------------

Company: hindustanunilever
Target Price:  2779.1175595044224
Entry Price:  2084.338169628317
LTP:  2448.65
What if we buy now?  13.495908337427656
----------------------------------

icicibank
======>>>>>> ERROR <<<<<<=======
----------------------------------

indusindbank
======>>>>>> ERROR <<<<<<=======
----------------------------------

Company: infosys
Target Price:  1448.3234556322857
Entry Price:  1086.2425917242142
LTP:  1574.2
What if we buy now?  -7.996223120805129
-----------

In [20]:
final_df = pd.read_csv('ev_ebitda_scraper.csv')
final_df

,company,target_price,entry_price,ltp,percent_increase_if_buy_now
0,adaniportsspecialeconomiczone,815.54,611.65,712.20,14.51
1,asianpaints,2911.96,2183.97,3003.90,-3.06
2,bajajauto,3701.75,2776.31,4196.15,-11.78
3,bajajfinance,7746.09,5809.57,6075.80,27.49
4,bajajfinserv,13495.38,10121.54,12486.60,8.08
5,bhartiairtel,556.69,417.52,535.10,4.03
6,bharatpetroleumcorporation,571.51,428.63,470.05,21.58
7,britanniaindustries,4401.12,3300.84,3670.05,19.92
8,cipla,896.77,672.58,957.10,-6.30
9,coalindia,142.11,106.58,148.75,-4.46


In [21]:
final_df.sort_values(by='percent_increase_if_buy_now', ascending=False)

,company,target_price,entry_price,ltp,percent_increase_if_buy_now
21,indianoilcorporation,187.30,140.48,111.40,68.14
27,ntpc,178.88,134.16,116.05,54.14
15,housingdevelopmentfinancecorporation,3812.89,2859.67,2509.20,51.96
3,bajajfinance,7746.09,5809.57,6075.80,27.49
6,bharatpetroleumcorporation,571.51,428.63,470.05,21.58
39,ultratechcement,8310.16,6232.62,6923.00,20.04
7,britanniaindustries,4401.12,3300.84,3670.05,19.92
31,shreecements,34402.03,25801.52,28729.10,19.75
28,oilnaturalgascorporation,143.56,107.67,120.90,18.74
30,relianceindustries,2420.18,1815.13,2104.45,15.00
